# Final Project University of Nottingham Final Exam Scheduling

### Team members: Kien Le, Jenny Nguyen, Dionne Phan
### Instructor: Dr. Bonnifonte

## I - Introduction and motivation

During final exam season (including the time when we wrote this report), it is common to see students expressing stress and frustration about their tightly packed schedules. As one student shared, "My exam schedule is terrible. I have to take three exams in a row from morning to night, and then turn in my final project at 10 a.m. the next morning" (Kien during his sophomore year). Experiences like this are common at Denison, and they highlight a central concern: **why do exam schedules end up being so stressful, and could they be designed more fairly?**

This project addresses that concern by examining the principles behind final exam schedules and the challenges that limit how balanced they can be. Our goal is to understand the basic structure of the exam scheduling problem and determine whether seemingly "bad" schedules are the result of randomness, unavoidable constraints, or flaws in the scheduling model itself. While we initially hoped to analyze Denison data, that information was not available (and we know that another group had already conducted a similar project). Instead, we used the **University of Nottingham’s final exam schedule** as a representative dataset, since it reflects the typical constraints faced by universities—such as exam room capacity, overlapping student enrollments, departmental requirements, fixed exam periods, and other complex considerations.

From this dataset, we set out to answer four central research questions:

1. **What is the definition of a "good" exam schedule for students?**
2. **How can an exam schedule be constructed in a systematic and principled way?**
3. **What are the key bottlenecks or limiting factors that make exam scheduling difficult?**
4. **If we build a working scheduling model, how can it be applied or adapted to Denison’s exam system?**

Understanding these questions is valuable not only for students, but also for administrators responsible for producing exam schedules. Insights from this analysis may reveal whether better scheduling algorithms could lead to a better exam experiences, or whether certain bottlenecks are simply unavoidable.

This report is organized as follows. Section 2 presents the preliminaries, including our data sources, data-wrangling process, and the overall setup of the problem. Section 3 details the modeling steps we used to construct and evaluate exam schedules, followed by an analysis of the results. Finally, Section 4 discusses how our model could be applied to Denison’s exam scheduling system and concludes with the broader implications of our findings.

## II - Preliminaries

In this section, we introduce the data sources and structure of the dataset, outline the key objectives guiding our exam scheduling model, and describe our preprocessing steps and setup required for model construction,.

### a. Data Source and Structure

The [exam timetabling dataset](https://people.cs.nott.ac.uk/pszrq/data.htm) was obtained from the University of Nottingham and is publicly available for research purposes. It was originally introduced by Burke, Newall, and Weare (1996) in their foundational study on university examination scheduling and has since become a standard benchmark within the exam timetabling community. The dataset represents real scheduling data of the 1994 - 1995 academic year at the University of Nottingham and includes detailed information on students, courses, exams, and enrolments. In total, the dataset contains 800 exams across 46 departments, 7,896 students, and 33,997 enrollments—equivalent to an average of 4.3 exams per student and 42.5 students per exam. A key feature of the dataset is its global room-capacity constraint: in any given timeslot, the total number of students taking exams cannot exceed 1,550. The primary objective defined in the original study is to minimize the number of students assigned to two consecutive exams on the same day. A later extension expands the objective to also avoid consecutive overnight exams (Burke & Carter, 1998).

Because it is based on real institutional data, incorporates practical scheduling constraints, and defines clear optimization objectives, the University of Nottingham dataset provides a rigorous, transparent, and reproducible benchmark for evaluating exam scheduling models. The dataset is provided in text format and consists of four main files:

1. `students`: This file contains information on student enrollment in courses. It has two columns:
* Student code: a 10-character unique identifier for each student.
* Course code: the code of the course the student is enrolled in.

1. `exams`: This file provides details about each exam and consists of four columns:
* Exam code: a unique identifier for the exam.
* Course name: the name of the course associated with the exam.
* Exam duration: the length of the exam in minutes.
* Department code: the code of the department responsible for the exam.

1. `enrolments`: This file links the students and exams datasets. It contains two columns:
* Student code: the unique identifier for a student.
* Exam code: the unique identifier for an exam in which the student is enrolled.

1. `data`: This file is not a dataset but specifies the necessary conditions and constraints for course scheduling, such as room capacities, allowable time slots, and other parameters and special constraints required for constructing the timetable.

### b. Model objectives 

Our model is designed to achieve the following objectives:

1. **Construct a feasible exam schedule** that satisfies all required constraints, including room capacity limits, departmental requirements, and student enrollment.

2. **Minimize the number of students assigned to two consecutive exams on the same day**.

3. **Minimize overnight consecutive exams**, ensuring that students have adequate time to rest and prepare between assessment periods.

Given these three objectives, our modeling procedure is designed to achieve them step by step. This structured approach not only guides the construction of a feasible timetable but also helps identify the key bottlenecks and challenges that arise during exam scheduling. Taken together, these objectives aim to produce an exam schedule that is both logistically sound and student-centered, balancing operational feasibility with consideration for student well-being.

In [2]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
from pathlib import Path
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS, FixedTicker, LabelSet
from bokeh.layouts import column
output_notebook()

Loading BokehJS ...

In [3]:
ROOT = Path().resolve().parent

# load data
enrolments = pd.read_csv(ROOT / "converted-data" / "enrolments.csv")
exams = pd.read_csv(ROOT / "converted-data" / "exams.csv")
students = pd.read_csv(ROOT / "converted-data" / "students.csv")
coincidences = pd.read_csv(ROOT / "converted-data" / "coincidences.csv", index_col=0)
dates = pd.read_csv(ROOT / "converted-data" / "dates.csv")
times_expanded = pd.read_csv(ROOT / "converted-data" / "times_expanded.csv")
rooms = pd.read_csv(ROOT / "converted-data" / "rooms.csv")
earliness = pd.read_csv(ROOT / "converted-data" / "earliness.csv")

In [4]:
# Helper function: convert exam duration from hh:mm to minutes
def parse_duration(time_str):
    h, m = time_str.split(":")
    return int(h) * 60 + int(m)

# Converting hours to mins
exams["duration_minutes"] = exams["duration"].apply(parse_duration)
times_expanded["timeslot"] = range(1, len(times_expanded) + 1)
times_expanded["slot_minutes"] = times_expanded["duration_hours"] * 60

In [5]:
# exam_list (list): the list of every exam
exam_list = exams["exam"].unique().tolist()

# duration (dict): the mapping from exam -> duration (mins)
duration = exams.set_index("exam")["duration_minutes"].to_dict()

# timeslots (list): the list of every timeslot
timeslots = times_expanded["timeslot"].tolist()

# slot_length (dict): the mapping from timeslots -> timeslots length (mins)
slot_length = times_expanded.set_index("timeslot")["slot_minutes"].to_dict()

# timeslot_day (dict): the mapping from timeslots -> day it occurs, in the form day_week
timeslot_day = {
    int(row["timeslot"]): f"{row['day']}_{row['week']}"
    for _, row in times_expanded.iterrows()
}

# days (list): the list of exam day, in the form day_week
days = list(set(timeslot_day.values()))

# day_to_timeslots (dict): the mapping from days -> list of timeslots in that day (sorted)
day_to_timeslots = {}
for t, day in timeslot_day.items():
    day_to_timeslots.setdefault(day, []).append(t)
for d in day_to_timeslots:
    day_to_timeslots[d] = sorted(day_to_timeslots[d])

am_slots = [day_to_timeslots[d][0] for d in day_to_timeslots.keys()]


# students_exams (dict): the mapping from students -> list of exams to take
students_exams = enrolments.groupby("student")["exam"].apply(list).to_dict()

# room_list (list): the list of every available room
room_list = rooms["room"].tolist()

# cap (dict): the mapping from room_list -> capacity of that room
cap = rooms.set_index("room")["capacity"].to_dict()

# size (dict): the mapping from exams -> number of students in that exam
size = (
    enrolments.groupby("exam")["student"]
    .nunique()
    .sort_values(ascending=False)
    .to_dict()
)

# groups (dict): the mapping from index -> lists of coincidences exam
groups = {
    k: ast.literal_eval(v) for k, v in zip(coincidences.index, coincidences["exams"])
}

# allow_pairs (list): the list of list of coincidences pairs
allowed_pairs = [
    (e1, e2)
    for exam_list_g in groups.values()
    for i, e1 in enumerate(exam_list_g)
    for e2 in exam_list_g[i + 1 :]
    for _ in range(2)
][::-1][::2][::-1] + [
    (e2, e1)
    for exam_list_g in groups.values()
    for i, e1 in enumerate(exam_list_g)
    for e2 in exam_list_g[i + 1 :]
]

# together_groups (list): the list of list of merge-able rooms
together_groups = [
    ["POPE-A13", "POPE-A14"],
    ["ART-LECTURE", "ART-SEMINAR"],
    ["SPORT-LGE1", "SPORT-LGE2"],
]

# special_exams (list): the list of exams with special rooms
special_exams = ["F12X02E1", "K1AHWAE2", "V13101E1"]

# super_rooms (list): the list of index of super rooms
super_rooms = list(range(len(together_groups)))

# super_capacity (dict): the mapping from super rooms index -> its capacity
super_capacity = {g: sum(cap[r] for r in together_groups[g]) for g in super_rooms}

# super_to_rooms (dict): the mapping from super rooms index -> the list of rooms that merge into
super_to_rooms = {g: together_groups[g] for g in super_rooms}

# time_order (list): list of available timeslot per day
time_order = ["9:00", "13:30", "16:30"]

# max_capacity (const): maximum students per timeslot
max_capacity = 1550

# earliness_priority (dict): the mapping from exam lists -> its priority. Higher priority means earlier (if possible)
earliness_priority = (earliness.set_index("exam")["priority"].to_dict()| {e: 0 for e in exam_list if e not in earliness["exam"].values})

# time_to_y (dict): the mapping from time strings -> numeric Y values (for plotting)
time_to_y = {t: i + 1 for i, t in enumerate(time_order)}

# day_to_x (dict): the mapping from days -> numeric X values (for plotting)
day_to_x = {"Mon": 1, "Tue": 2, "Wed": 3, "Thu": 4, "Fri": 5, "Sat": 6}

am_slots


[1, 4, 7, 10, 13, 16, 17, 20, 23, 26, 29, 32]

In [6]:
def plot_source(schedule_df):
    """
    Convert the given schedule to plot source
    schedule_df: the schedule df with the columns [exam, day, week, start_time, duration_min, timeslot]
    Output: plot source for plotting
    """
    x_coords, y_coords, exam_texts, week_list = [], [], [], []

    for _, row in schedule_df.iterrows():
        day = row["day"]
        time = row["start_time"]
        week = row["week"]

        y_val = time_to_y[time] + 4 * (week - 1)
        x_coords.append(day_to_x[day])
        y_coords.append(y_val)
        exam_texts.append(row["exam"])
        week_list.append(week)

    student_counts = [size[e] for e in exam_texts]

    # Combine exams in same cell
    df_vis = pd.DataFrame(
        {
            "day": x_coords,
            "y": y_coords,
            "week": week_list,
            "exam": exam_texts,
            "students": student_counts,
        }
    )

    df_grouped = (
        df_vis.groupby(["day", "y", "week"])
        .agg({"exam": lambda x: list(x), "students": "sum"})
        .reset_index()
    )
    df_grouped["exam_count"] = df_grouped["exam"].apply(len)

    df_grouped["label"] = df_grouped.apply(
        lambda r: f"{r['exam_count']} exams \n ({r['students']} students)", axis=1
    )
    source = ColumnDataSource(df_grouped)
    return source


def make_plot(source, plot_name):
    """
    Plot the given plot source
    source: Plot source
    plot_name: Plot name
    """
    p = figure(
        x_range=(0.5, 6.5),
        y_range=[8, 0],
        height=600,
        title=plot_name,
        tools="tap",
    )
    p.yaxis.ticker = FixedTicker(ticks=[1, 2, 3, 5, 6, 7])
    p.ygrid.ticker = FixedTicker(ticks=[1, 2, 3, 5, 6, 7])
    p.yaxis.major_label_overrides = {
        1: "9:00",
        2: "13:30",
        3: "16:30",
        5: "9:00",
        6: "13:30",
        7: "16:30",
    }
    p.xaxis.ticker = [1, 2, 3, 4, 5, 6]
    p.xaxis.major_label_overrides = {
        1: "Mon",
        2: "Tue",
        3: "Wed",
        4: "Thu",
        5: "Fri",
        6: "Sat",
    }
    p.xaxis.axis_label = "Day"
    p.yaxis.axis_label = "Start Time"
    p.xaxis.major_label_orientation = 1.2

    p.rect(
        x="day",
        y="y",
        width=0.9,
        height=0.8,
        source=source,
        fill_color="skyblue",
        line_color="black",
    )

    p.text(
        x="day",
        y="y",
        text="label",
        source=source,
        text_align="center",
        text_baseline="middle",
        text_font_size="8pt",
    )

    # Callback
    callback = CustomJS(
        args=dict(source=source),
        code="""
        const data = source.data;
        const idx = source.selected.indices[0];
        if(idx !== undefined){
            const day = data['day'][idx];
            const week = data['week'][idx];
            const y = data['y'][idx];
            const exams = data['exam'][idx];

            alert(
                'Day: ' + day +
                '\\nWeek: ' + week +
                '\\nTime slot: ' + y +
                '\\nExams: ' + exams
            );
        }
    """,
    )

    source.selected.js_on_change("indices", callback)

    show(p)

### c. Data Preprocessing and Setup

Before building the model, we performed data wrangling and set up the variables that will be used in the subsequent modeling steps. The key steps we did on data wrangling are as follows:

1. Convert dataset formats: The original text files were transformed into CSV format to facilitate easier data access and manipulation.
2. Merge datasets and extract variables: Relevant information from the students, exams, enrolments, and data files was combined, and the variables required for modeling were extracted.

The variables will be used in the next modeling section are defined as follows:

* `exam_list` *(List)*: A list containing all exam codes.
* `timeslots` *(List)*: A list of available timeslots, enumerated from 1 to 32.
* `students_exams` *(Dictionary)*: A mapping where each key is a student code, and the corresponding value is the list of exams that the student is required to take, drawn from the `exam_list`.
* `durations` *(Dictionary)*: A mapping where each key is an exam code, and the corresponding value is the exam length (in minutes).
* `size` *(Dictionary)*: A mapping where each key is an exam code, and the corresponding value is the number of students enrolls in that exam.

In addition, visualization methods were also designed to display the results of the generated schedules. These visualizations help to understand the structure and quality of the solutions and support easier interpretation of the outcomes.

## III - Modelling

In this section, we will iteratively build the exam scheduling models, gradually adding constraints from one model to the next. 

### Model 1: A basic feasible exam timetabling model
This initial model, while simple and not fully constrained, is presented in detail to establish a foundation for understanding the subsequent models. Thus, the repetitive and tedious variables and constraints in the later models will be omitted. We will instead focus on explaining the key constraints and modifications introduced at each stage.

We first define the following sets:

- $C$: the set of exams.
- $T$: the set of available timeslots.
- $S$: the set of students.
- $E_s \subseteq C$: the set of exams taken by student $s \in S$.

We will next define our decision variables $x_{i,t}$ for $i \in C$ and $t \in T$ as

$$
x_{i,t} =
\begin{cases}
1, & \text{if exam } i \text{ is scheduled in timeslot } t, \\
0, & \text{otherwise}.
\end{cases}
$$

For this model, we apply the following constraints:

1. Every exam is assigned to one and only one timeslot, which means
$$
\sum_{t \in T} x_{i,t} = 1 \qquad \forall i \in C.
$$

2. For every student $s$ and for every pair of exams $(i, j \in E_s)$ with $(i \neq j)$, they cannot be assigned to the same timeslot:
$$
x_{i,t} + x_{j,t} \le 1
\qquad \forall s \in S,\; \forall i \neq j \in E_s,\; \forall t \in T.
$$

3. Exam duration must not exceed timeslot length. Let $\text{dur}_i$ be the duration of exam $i$, and let $\text{len}_t$ be the allowable duration of timeslot $t$. If an exam is too long to fit into a timeslot, the assignment is forbidden:

$$
x_{i,t} = 0
\qquad \forall i \in C,\; \forall t \in T \text{ such that } \text{dur}_i > \text{len}_t.
$$

For this model, our aim is just to obtain a feasible schedule. Thus, we may use a dummy objective:

$$
\min 0
$$

or equivalently:

$$
\text{Find any feasible assignment satisfying all constraints.}
$$

The implementation of this model is the following

In [61]:
# MODEL 1
model1 = gp.Model("Basic feasible exam timetabling model")
model1.setParam("OutputFlag", 0)

# Binary vars x[e,t]
x = model1.addVars(exam_list, timeslots, vtype=GRB.BINARY, name="x")

# 1. Each exam must be scheduled exactly once
for e in exam_list:
    model1.addConstr(
        gp.quicksum(x[e, t] for t in timeslots) == 1, name=f"assign_once_{e}"
    )

# 2. No student may have two exams in the same timeslot
for s, exam_list_s in students_exams.items():
    if len(exam_list_s) > 1:
        for t in timeslots:
            for i in range(len(exam_list_s)):
                for j in range(i + 1, len(exam_list_s)):
                    e1 = exam_list_s[i]
                    e2 = exam_list_s[j]
                    model1.addConstr(
                        x[e1, t] + x[e2, t] <= 1, name=f"no_overlap_{s}_{e1}_{e2}_{t}"
                    )

# 3. Exam duration <= timeslot duration
for e in exam_list:
    for t in timeslots:
        if duration[e] > slot_length[t]:
            model1.addConstr(x[e, t] == 0, name=f"forbidden_duration_{e}_{t}")

# 4. Max capacity
for t in timeslots:
    model1.addConstr(
        gp.quicksum(size[e] * x[e, t] for e in exam_list) <= max_capacity,
        name=f"timeslot_capacity_{t}"
    )

# Objective: Feasible schedule
model1.setObjective(0, GRB.MINIMIZE)

# Solve model
model1.optimize()

In [7]:
def get_schedule_model1(x):
    """
    Extracts the exam schedule from the solution of Model 1 and converts it into a DataFrame
    x: Decision variables
    Output: A table containing the scheduled exams
    """
    schedule_rows = []

    for e in exam_list:
        for t in timeslots:
            if x[e, t].X > 0.5:
                row = times_expanded[times_expanded["timeslot"] == t].iloc[0]
                schedule_rows.append(
                    {
                        "exam": e,
                        "day": row["day"],
                        "week": row["week"],
                        "start_time": row["start_time"],
                        "duration_min": row["slot_minutes"],
                        "timeslot": t,
                    }
                )

    schedule_df = pd.DataFrame(schedule_rows)
    schedule_df.sort_values(["timeslot"], inplace=True)
    return schedule_df

In [63]:
# The exam timetable for Model 1
schedule_df = get_schedule_model1(x)
source1 = plot_source(schedule_df)
make_plot(source1, "Exam Timetable(Model 1)")

### Model 2: Coincidences

In [64]:
# MODEL 2
model2 = gp.Model("Model with coincidences")
model2.setParam("OutputFlag", 0)

# Binary vars x[e,t]
x = model2.addVars(exam_list, timeslots, vtype=GRB.BINARY, name="x")

# Binary vars c[g,t]
c = model2.addVars(groups.keys(), timeslots, vtype=GRB.BINARY, name="c")

# 1. Each exam must be scheduled exactly once
for e in exam_list:
    model2.addConstr(
        gp.quicksum(x[e, t] for t in timeslots) == 1, name=f"assign_once_{e}"
    )

# 2. No student may have two exams in the same timeslot
for s, exam_list_s in students_exams.items():
    for i in range(len(exam_list_s)):
        for j in range(i + 1, len(exam_list_s)):
            e1, e2 = exam_list_s[i], exam_list_s[j]
            if (e1, e2) in allowed_pairs:
                continue  # skip, they are in a coincidence group
            for t in timeslots:
                model2.addConstr(
                    x[e1, t] + x[e2, t] <= 1, name=f"no_overlap_{s}_{e1}_{e2}_{t}"
                )

# 3. Exam duration <= timeslot duration
for e in exam_list:
    for t in timeslots:
        if duration[e] > slot_length[t]:
            model2.addConstr(x[e, t] == 0, name=f"forbidden_duration_{e}_{t}")

# 4. c[g,t] is the same as x[e,t]
for g, exam_list_g in groups.items():
    for e in exam_list_g:
        for t in timeslots:
            model2.addConstr(x[e, t] <= c[g, t], name=f"link_exam_to_group_{g}_{e}_{t}")

# 5. Coincidences must be scheduled in the same timeslot
for g in groups:
    model2.addConstr(
        gp.quicksum(c[g, t] for t in timeslots) == 1, name=f"group_once_{g}"
    )

# 6. Max capacity
for t in timeslots:
    model2.addConstr(
        gp.quicksum(size[e] * x[e, t] for e in exam_list) <= max_capacity,
        name=f"timeslot_capacity_{t}",
    )

# Objective: Feasible schedule
model2.setObjective(0, GRB.MINIMIZE)

# Solve model
model2.optimize()

In [65]:
# The exam timetable for Model 2
schedule_df = get_schedule_model1(x)
source2 = plot_source(schedule_df)
make_plot(source2, "Exam Timetable(Model 2)")

### Model 3: Room assignment

This model will next introduce the room size constraints for the model. That is, the number of students in each exam must not exceed the capacity of the room that one is assigned for. To incorporate room capacity constraints into the model, we introduce a new set $R$, representing the set of available exam rooms. For each room $r \in R$, we define $\text{cap}_r$ as its capacity. Similarly, for each exam $i \in C$, we denote $\text{size}_i$ as the number of students enrolled in that exam.

However, this constraint alone may result in an infeasible model because some exams have more students than any single room can accommodate. To address this issue, we allow certain rooms to be merged into a larger "super room" so that they can jointly host an oversized exam. Let $G$ be the set of all permitted super rooms. Each super room $G_g \in G$ is a subset of rooms $G_g \subseteq R$, that can be combined and used simultaneously as a single space. The capacity of a super room is defined as

$$
\text{cap}_g = \sum_{r \in G_g} \text{cap}_r.
$$

When a super room is used, all rooms in $G_g$ are considered occupied by that exam during the assigned time slot. Consequently, no two different exams may use any of the same rooms (or super rooms composed of overlapping rooms) at the same time to avoid room conflicts.

We introduce two additional sets of binary decision variables to model the use of individual rooms and super rooms. First, for each exam $i \in C$, time slot $t \in T$, and room $r \in R$, we define:

$$
z_{i,t,r} =
\begin{cases}
1 & \text{if exam } i \text{ is assigned to room } r \text{ at time slot } t, \\
0 & \text{otherwise}.
\end{cases}
$$

Second, for each exam $i \in C$, time slot $t \in T$, and super room $g \in G$, we define:

$$
y_{i,t,g} =
\begin{cases}
1 & \text{if exam } i \text{ uses super room } g \text{ at time slot } t, \\
0 & \text{otherwise}.
\end{cases}
$$

To enforce the correct use of individual rooms and super rooms, several additional constraints are required.

1. Linking constraint: An exam can only occupy a room or a super room if it is scheduled in that time slot. For individual rooms, we have

$$
z_{i,t,r} \le x_{i,t}
\qquad \forall i \in C, t \in T, r \in R,
$$

and similarly for super rooms

$$
y_{i,t,g} \le x_{i,t}
\qquad \forall i \in C, t \in T, g \in G.
$$

2. Occupancy constraint: If an exam uses a super room $g$, then all rooms that make up that super room are occupied. Thus, for every room $r \in G_g$:

$$
z_{i,t,r} \le 1 - y_{i,t,g}
\qquad \forall g \in G, r \in G_g, \forall i \in C, t \in T.
$$

3. The rooms (or super room) assigned to an exam must provide sufficient seating for all students enrolled in that exam. The combined capacity of the assigned rooms must be at least the exam size, so we have

$$
\sum_{r \in R} \text{cap}_r z_{i,t,r} + \sum_{g \in G} \text{cap}_g y_{i,t,g} \ge \text{size}_i x_{i,t} \qquad \forall i \in C, t \in T.
$$

4. A room cannot host more than one exam in the same time slot. Thus

$$
\sum_{i \in C} z_{i,t,r} \le 1 \qquad \forall t \in T, r \in R.
$$


The implementation of model 3 is the following

In [66]:
# MODEL 3
model3 = gp.Model("Model with room assignments")
model3.setParam("OutputFlag", 0)

# Binary vars x[e,t]
x = model3.addVars(exam_list, timeslots, vtype=GRB.BINARY, name="x")

# Binary vars c[g,t]
c = model3.addVars(groups.keys(), timeslots, vtype=GRB.BINARY, name="c")

# Binary vars z[e,t,r]
z = model3.addVars(exam_list, timeslots, room_list, vtype=GRB.BINARY, name="z")

# Integer vars p[e,t,r]
p = model3.addVars(exam_list, timeslots, room_list, vtype=GRB.INTEGER, lb=0, name="p")

# 1. Each exam must be scheduled exactly once
for e in exam_list:
    model3.addConstr(gp.quicksum(x[e, t] for t in timeslots) == 1)

# 2. No student may have two exams in the same timeslot
for s, exam_list_s in students_exams.items():
    for i in range(len(exam_list_s)):
        for j in range(i + 1, len(exam_list_s)):
            e1, e2 = exam_list_s[i], exam_list_s[j]
            if (e1, e2) in allowed_pairs:
                continue  # skip, they are in a coincidence group
            for t in timeslots:
                model3.addConstr(
                    x[e1, t] + x[e2, t] <= 1, name=f"no_overlap_{s}_{e1}_{e2}_{t}"
                )

# 3. Exam duration <= timeslot duration
for e in exam_list:
    for t in timeslots:
        if duration[e] > slot_length[t]:
            model3.addConstr(x[e, t] == 0, name=f"forbidden_duration_{e}_{t}")

# 4. c[g,t] is the same as x[e,t]
for g, exam_list_g in groups.items():
    for e in exam_list_g:
        for t in timeslots:
            model3.addConstr(x[e, t] <= c[g, t], name=f"link_exam_to_group_{g}_{e}_{t}")

# 5. Coincidences must be scheduled in the same timeslot
for g in groups:
    model3.addConstr(
        gp.quicksum(c[g, t] for t in timeslots) == 1, name=f"group_once_{g}"
    )

# 6. Max capacity
for t in timeslots:
    model3.addConstr(
        gp.quicksum(size[e] * x[e, t] for e in exam_list) <= max_capacity,
        name=f"timeslot_capacity_{t}",
    )

# 7. A room may be used only if exam is scheduled in that timeslot
for e in exam_list:
    for t in timeslots:
        for r in room_list:
            model3.addConstr(z[e, t, r] <= x[e, t])

# 8. Students assign to a room must be smaller than its capacity
for e in exam_list:
    for t in timeslots:
        for r in room_list:
            model3.addConstr(p[e, t, r] <= cap[r] * z[e, t, r])

# 9. Total room capacity
for t in timeslots:
    for r in room_list:
        model3.addConstr(gp.quicksum(p[e, t, r] for e in exam_list) <= cap[r])

# 10. Total exam size
for e in exam_list:
    for t in timeslots:
        model3.addConstr(
            gp.quicksum(p[e, t, r] for r in room_list) == size[e] * x[e, t]
        )

# 11. Split an exam into no more than 3 room
K = 3  # max 3 rooms per exam
for e in exam_list:
    for t in timeslots:
        model3.addConstr(gp.quicksum(z[e, t, r] for r in room_list) <= K)

# Objective: Feasible schedule
model3.setObjective(
    0, GRB.MINIMIZE
)  # minimize room spliting will takes much longer, also no point to do it

# Solve model
model3.optimize()

In [10]:
def get_schedule_model2(x, z):
    """
    Extract the exam schedule including room assignments.

    x[e,t] = 1 if exam e is scheduled in timeslot t
    z[e,t,r] = 1 if exam e uses room r in timeslot t
    """

    schedule_rows = []

    for e in exam_list:
        for t in timeslots:
            if x[e, t].X > 0.5:
                row = times_expanded[times_expanded["timeslot"] == t].iloc[0]
                rooms_used = [r for r in room_list if z[e, t, r].X > 0.5]
                schedule_rows.append(
                    {
                        "exam": e,
                        "day": row["day"],
                        "week": row["week"],
                        "start_time": row["start_time"],
                        "duration_min": row["slot_minutes"],
                        "timeslot": t,
                        "rooms": rooms_used,
                    }
                )

    schedule_df = pd.DataFrame(schedule_rows)
    schedule_df.sort_values(["timeslot", "exam"], inplace=True)
    return schedule_df


def get_room_usage_df(x, z):
    """
    Build a table of room usage.
    Each row: (timeslot, room)
    Column: list of exams in that room at that timeslot
    """
    rows = []

    for t in timeslots:
        for r in room_list:
            exams_in_room = []
            for e in exam_list:
                if z[e, t, r].X > 0.5:
                    exams_in_room.append(e)
            rows.append({"timeslot": t, "room": r, "exams": exams_in_room})

    df = pd.DataFrame(rows)
    df.sort_values(["timeslot", "room"], inplace=True)
    return df

In [68]:
schedule_df = get_schedule_model2(x, z)
source3 = plot_source(schedule_df)
rooms_usage_df = get_room_usage_df(x,z)
make_plot(source3, "Exam Timetable(Model 3)")

In [69]:
# MODEL 4:
model4 = gp.Model("Model with gap constraint")
# model4.setParam("OutputFlag", 0)

# Binary vars x[e,t]
x = model4.addVars(exam_list, timeslots, vtype=GRB.BINARY, name="x")

# Binary vars c[g,t]
c = model4.addVars(groups.keys(), timeslots, vtype=GRB.BINARY, name="c")

# Binary vars  z[e,t,r]
z = model4.addVars(exam_list, timeslots, room_list, vtype=GRB.BINARY, name="z")

# Integer vars p[e,t,r]
p = model4.addVars(exam_list, timeslots, room_list, vtype=GRB.INTEGER, lb=0, name="p")

# 1. Each exam must be scheduled exactly once
for e in exam_list:
    model4.addConstr(
        gp.quicksum(x[e, t] for t in timeslots) == 1, name=f"exam_once_{e}"
    )

# 2. No student may have two exams in the same timeslot
for s, exams_s in students_exams.items():
    for i in range(len(exams_s)):
        for j in range(i + 1, len(exams_s)):
            e1, e2 = exams_s[i], exams_s[j]
            if (e1, e2) in allowed_pairs:
                continue
            for t in timeslots:
                model4.addConstr(
                    x[e1, t] + x[e2, t] <= 1, name=f"no_overlap_{s}_{e1}_{e2}_{t}"
                )

# 3. Exam duration <= timeslot duration
for e in exam_list:
    for t in timeslots:
        if duration[e] > slot_length[t]:
            model4.addConstr(x[e, t] == 0, name=f"forbidden_duration_{e}_{t}")

# 4. Link exams to coincidence group
for g, exams_g in groups.items():
    for e in exams_g:
        for t in timeslots:
            model4.addConstr(x[e, t] <= c[g, t], name=f"link_exam_to_group_{g}_{e}_{t}")

# 5. Coincidence group scheduled exactly once
for g in groups:
    model4.addConstr(
        gp.quicksum(c[g, t] for t in timeslots) == 1, name=f"group_once_{g}"
    )

# 6. Max total capacity per timeslot
for t in timeslots:
    model4.addConstr(
        gp.quicksum(size[e] * x[e, t] for e in exam_list) <= max_capacity,
        name=f"timeslot_capacity_{t}",
    )

# 7. Rooms can be used only if exam is scheduled
for e in exam_list:
    for t in timeslots:
        for r in room_list:
            model4.addConstr(z[e, t, r] <= x[e, t], name=f"room_usage_link_{e}_{t}_{r}")

# 8. Students assigned to a room <= room capacity
for e in exam_list:
    for t in timeslots:
        for r in room_list:
            model4.addConstr(
                p[e, t, r] <= cap[r] * z[e, t, r],
                name=f"room_capacity_link_{e}_{t}_{r}",
            )

# 9. Total exam size distributed across rooms
for e in exam_list:
    for t in timeslots:
        model4.addConstr(
            gp.quicksum(p[e, t, r] for r in room_list) == size[e] * x[e, t],
            name=f"exam_size_{e}_{t}",
        )

# 10. Split exams into no more than 3 rooms
K = 3  # Maximum number of rooms
for e in exam_list:
    for t in timeslots:
        model4.addConstr(
            gp.quicksum(z[e, t, r] for r in room_list) <= K, name=f"max_rooms_{e}_{t}"
        )

# 11. Two exams on same day must have at least 1 gap
for s, exams_s in students_exams.items():
    for d, t_list in day_to_timeslots.items():
        if len(t_list) == 3:
            first_slot = t_list[0]
            middle_slot = t_list[1]
            last_slot = t_list[2]

            # For each pair of non-coincidence exams
            for i in range(len(exams_s)):
                for j in range(i + 1, len(exams_s)):
                    e1, e2 = exams_s[i], exams_s[j]
                    if (e1, e2) in allowed_pairs:
                        continue
                    model4.addConstr(
                        x[e1, middle_slot] + x[e2, first_slot] <= 1,
                        name=f"gap_{s}_{e1}_{e2}_{d}_mid_first",
                    )
                    model4.addConstr(
                        x[e1, middle_slot] + x[e2, middle_slot] <= 1,
                        name=f"gap_{s}_{e1}_{e2}_{d}_mid_mid",
                    )
                    model4.addConstr(
                        x[e1, middle_slot] + x[e2, last_slot] <= 1,
                        name=f"gap_{s}_{e1}_{e2}_{d}_mid_last",
                    )
                    model4.addConstr(
                        x[e1, first_slot] + x[e2, middle_slot] <= 1,
                        name=f"gap_{s}_{e1}_{e2}_{d}_first_mid",
                    )
                    model4.addConstr(
                        x[e1, last_slot] + x[e2, middle_slot] <= 1,
                        name=f"gap_{s}_{e1}_{e2}_{d}_last_mid",
                    )

# Objective: Feasible schedule
model4.setObjective(0, GRB.MINIMIZE)

# Solve model
model4.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6126754 rows, 845888 columns and 13045464 nonzeros
Model fingerprint: 0x958c7256
Variable types: 0 continuous, 845888 integer (436288 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 4618788 rows and 40000 columns (presolve time = 5s)...
Presolve removed 5194044 rows and 46032 columns
Presolve time: 5.57s
Presolved: 932710 rows, 799856 columns, 2920984 nonzeros
Variable types: 0 continuous, 799856 integer (419248 binary)
Performing another presolve...
Presolve removed 35459 rows and 37440 columns (presolve time = 6s)...
Presolve removed 61435 rows and 37440 columns
Presolve time: 5.67s
Deterministic concurrent LP optimizer: primal simplex, dual simplex

In [70]:
def get_students_with_multiple_exams_per_day(model, exam_count, penalty_indicator, days):
    """
    Returns a DataFrame showing students with more than 1 exam per day.
    
    Columns: student, day, exam_count, multiple_exams (True/False)
    """
    rows = []
    for s in students_exams.keys():
        for d in days:
            count = int(exam_count[s,d].X)
            multiple = penalty_indicator[s,d].X > 0.5
            rows.append({
                "student": s,
                "day": d,
                "exam_count": count,
                "multiple_exams": multiple
            })
    
    df = pd.DataFrame(rows)
    df.sort_values(["day", "student"], inplace=True)
    return df

In [71]:
# Ensure the model was solved to optimality/feasibility
if model4.status != GRB.OPTIMAL and model4.status != GRB.FEASIBLE:
    print("No feasible solution found.")
else:
    # Build the schedule from x[e,t] values
    schedule = {e: None for e in exam_list}
    for e in exam_list:
        for t in timeslots:
            if x[e, t].X > 0.5:
                schedule[e] = t
students_double_exam = 0

for s, exams_s in students_exams.items():
    for d, t_list in day_to_timeslots.items():
        # Count non-coincidence exams scheduled on this day
        count = 0
        for i in range(len(exams_s)):
            for j in range(i+1, len(exams_s)):
                e1, e2 = exams_s[i], exams_s[j]
                if (e1, e2) in allowed_pairs:
                    continue  # allowed, skip
                # Check if both are scheduled on this day
                t1, t2 = schedule[e1], schedule[e2]
                if t1 in t_list and t2 in t_list:
                    count += 1
        if count > 0:
            students_double_exam += 1

print("Number of students forced to take 2+ exams on the same day (non-coincidence):", students_double_exam)


Number of students forced to take 2+ exams on the same day (non-coincidence): 2207


In [72]:
schedule_df = get_schedule_model2(x, z)
source3 = plot_source(schedule_df)
make_plot(source3, "Exam Timetable(Model 4)")

rooms_usage_df = get_room_usage_df(x,z)
schedule_df

,exam,day,week,start_time,duration_min,timeslot,rooms
8,AAB025E1,Mon,1,9:00,180,1,[COPSE-6]
18,B31103E1,Mon,1,9:00,180,1,[TRENT-HALL]
31,B33563E1,Mon,1,9:00,180,1,[SPORT-SMALL]
41,C12332E1,Mon,1,9:00,180,1,[SPORT-SMALL]
46,C13564E1,Mon,1,9:00,180,1,"[COPSE-4, COPSE-5]"
...,...,...,...,...,...,...,...
673,R81022E1,Sat,2,9:00,180,32,[SPORT-SMALL]
700,V12127E1,Sat,2,9:00,180,32,[SPORT-SMALL]
747,V61110E1,Sat,2,9:00,180,32,[SPORT-LGE1]
754,V6B308E1,Sat,2,9:00,180,32,[COPSE-4]


### Minor institutional constraints ("must" constraints)

Subset of timeslots
- F321Q6E1 must be at 27th Jan (Fri_1) Done
- F321T6E1 must be 30th Jan (Mon_2) Done
- H21M01E1, H22M02E1, H2CM04E1 must be 23rd-24th Jan (Mon_1, Tue_1)
- G13RE2E1  before 30th Jan (Week 1) => Slot 1-16
- K1AHWAE2, H63122E1 any am slot 
- V13101E1  any Thursday pm slot (slot) -> slot 11, 12, 27, 28

Timeslots relatives
F13P03E1 \ before / F13X03E1
F13P05E1 /        \ F13X04E1
H3BFM2E1  must be immediately followed by  H3BFM2E2
H8B040E1 \ must be at different times
H8C001E1 /
Q33211E1 \
Q33307E1 - spread out, mid-session (relaxing spreadout to be different time)
Q33308E1 /
B12301E1 \
B12302E1 - spread out (relaxing spreadout to be different time)
B12303E1 |
B12320E1 /

B13103E1              \ spread out (relaxing spreadout to be different time)
{C13571E1 & C13572E1} /


Room unavailable
Room TRENT-B46 unavailable morning of Friday 3rd Feb (not slot 29)

In [73]:
# MODEL 5
model5 = gp.Model("Model with miscellaneous")
# model5.setParam("OutputFlag", 0)

# Binary vars x[e,t]
x = model5.addVars(exam_list, timeslots, vtype=GRB.BINARY, name="x")

# Binary vars c[g,t]
c = model5.addVars(groups.keys(), timeslots, vtype=GRB.BINARY, name="c")

# Binary vars  z[e,t,r]
z = model5.addVars(exam_list, timeslots, room_list, vtype=GRB.BINARY, name="z")

# Integer vars p[e,t,r]
p = model5.addVars(exam_list, timeslots, room_list, vtype=GRB.INTEGER, lb=0, name="p")

# 1. Each exam must be scheduled exactly once
for e in exam_list:
    model5.addConstr(
        gp.quicksum(x[e, t] for t in timeslots) == 1, name=f"exam_once_{e}"
    )

# 2. No student may have two exams in the same timeslot
for s, exams_s in students_exams.items():
    for i in range(len(exams_s)):
        for j in range(i + 1, len(exams_s)):
            e1, e2 = exams_s[i], exams_s[j]
            if (e1, e2) in allowed_pairs:
                continue
            for t in timeslots:
                model5.addConstr(
                    x[e1, t] + x[e2, t] <= 1, name=f"no_overlap_{s}_{e1}_{e2}_{t}"
                )

# 3. Exam duration <= timeslot duration
for e in exam_list:
    for t in timeslots:
        if duration[e] > slot_length[t]:
            model5.addConstr(x[e, t] == 0, name=f"forbidden_duration_{e}_{t}")

# 4. Link exams to coincidence group
for g, exams_g in groups.items():
    for e in exams_g:
        for t in timeslots:
            model5.addConstr(x[e, t] <= c[g, t], name=f"link_exam_to_group_{g}_{e}_{t}")

# 5. Coincidence group scheduled exactly once
for g in groups:
    model5.addConstr(
        gp.quicksum(c[g, t] for t in timeslots) == 1, name=f"group_once_{g}"
    )

# 6. Max total capacity per timeslot
for t in timeslots:
    model5.addConstr(
        gp.quicksum(size[e] * x[e, t] for e in exam_list) <= max_capacity,
        name=f"timeslot_capacity_{t}",
    )

# 7. Rooms can be used only if exam is scheduled
for e in exam_list:
    for t in timeslots:
        for r in room_list:
            model5.addConstr(z[e, t, r] <= x[e, t], name=f"room_usage_link_{e}_{t}_{r}")

# 8. Students assigned to a room <= room capacity
for e in exam_list:
    for t in timeslots:
        for r in room_list:
            model5.addConstr(
                p[e, t, r] <= cap[r] * z[e, t, r],
                name=f"room_capacity_link_{e}_{t}_{r}",
            )

# 9. Total exam size distributed across rooms
for e in exam_list:
    for t in timeslots:
        model5.addConstr(
            gp.quicksum(p[e, t, r] for r in room_list) == size[e] * x[e, t],
            name=f"exam_size_{e}_{t}",
        )

# 10. Split exams into no more than 3 rooms
K = 3  # Maximum number of rooms
for e in exam_list:
    for t in timeslots:
        model5.addConstr(
            gp.quicksum(z[e, t, r] for r in room_list) <= K, name=f"max_rooms_{e}_{t}"
        )

# 11. Two exams on same day must have at least 1 gap
for s, exams_s in students_exams.items():
    for d, t_list in day_to_timeslots.items():
        if len(t_list) == 3:
            first_slot = t_list[0]
            middle_slot = t_list[1]
            last_slot = t_list[2]

            # For each pair of non-coincidence exams
            for i in range(len(exams_s)):
                for j in range(i + 1, len(exams_s)):
                    e1, e2 = exams_s[i], exams_s[j]
                    if (e1, e2) in allowed_pairs:
                        continue
                    model5.addConstr(
                        x[e1, middle_slot] + x[e2, first_slot] <= 1,
                        name=f"gap_{s}_{e1}_{e2}_{d}_mid_first",
                    )
                    model5.addConstr(
                        x[e1, middle_slot] + x[e2, middle_slot] <= 1,
                        name=f"gap_{s}_{e1}_{e2}_{d}_mid_mid",
                    )
                    model5.addConstr(
                        x[e1, middle_slot] + x[e2, last_slot] <= 1,
                        name=f"gap_{s}_{e1}_{e2}_{d}_mid_last",
                    )
                    model5.addConstr(
                        x[e1, first_slot] + x[e2, middle_slot] <= 1,
                        name=f"gap_{s}_{e1}_{e2}_{d}_first_mid",
                    )
                    model5.addConstr(
                        x[e1, last_slot] + x[e2, middle_slot] <= 1,
                        name=f"gap_{s}_{e1}_{e2}_{d}_last_mid",
                    )

# 12. Misc: F321Q6E1 must be scheduled on 27th Jan (Fri_1)
model5.addConstr(
    gp.quicksum(x["F321Q6E1", t] for t in day_to_timeslots["Fri_1"]) == 1,
    name="fixed_F321Q6E1_Fri_1"
)

# 13. Misc: F321T6E1 must be scheduled on 30th Jan (Mon_2)
model5.addConstr(
    gp.quicksum(x["F321T6E1", t] for t in day_to_timeslots["Mon_2"]) == 1,
    name="fixed_F321T6E1_Mon_2"
)

# 14. Misc: H21M01E1 must be scheduled on 23rd-24th Jan (Mon_1, Tue_1)
model5.addConstr(
    gp.quicksum(x["H21M01E1", t] for t in day_to_timeslots["Mon_1"] + day_to_timeslots["Tue_1"]) == 1,
    name="fixed_H21M01E1_Mon_1_Tue_1"
)

# 15. Misc: H22M02E1 must be scheduled on 23rd-24th Jan (Mon_1, Tue_1)
model5.addConstr(
    gp.quicksum(x["H22M02E1", t] for t in day_to_timeslots["Mon_1"] + day_to_timeslots["Tue_1"]) == 1,
    name="fixed_H22M02E1_Mon_1_Tue_1"
)

# 16. Misc: H2CM04E1 must be scheduled on 23rd-24th Jan (Mon_1, Tue_1)
model5.addConstr(
    gp.quicksum(x["H2CM04E1", t] for t in day_to_timeslots["Mon_1"] + day_to_timeslots["Tue_1"]) == 1,
    name="fixed_H2CM04E1_Mon_1_Tue_1"
)

# 17. Misc: G13RE2E1 must be scheduled before 30th Jan => timeslots 1-16
model5.addConstr(
    gp.quicksum(x["G13RE2E1", t] for t in timeslots[:16]) == 1,
    name="fixed_G13RE2E1_week1"
)

# 18. Misc: K1AHWAE2  must be scheduled in AM slots only
model5.addConstr(
    gp.quicksum(x["K1AHWAE2", t] for t in am_slots) == 1,
    name="fixed_K1AHWAE2_am"
)

# 19. Misc: H63122E1  must be scheduled in AM slots only
model5.addConstr(
    gp.quicksum(x["H63122E1", t] for t in am_slots) == 1,
    name="fixed_H63122E1_am"
)

# 20. Misc: V13101E1 must be scheduled in any Thursday PM slot (slots 11, 12, 27, 28)
thursday_pm_slots = [11, 12, 27, 28]
model5.addConstr(
    gp.quicksum(x["V13101E1", t] for t in thursday_pm_slots) == 1,
    name="fixed_V13101E1_thu_pm"
)

# 21. Misc: TRENT-B46 unavailable morning of Friday 3rd Feb (slot 29)
for e in exam_list:
    model5.addConstr(
        z[e, 29, "TRENT-B46"] == 0,
        name=f"room_unavail_{e}_29_TRENT-B46"
    )

# 22. Misc: F13P03E1 and F13P05E1 are before F13X03E1 and F13X04E1
precedence_pairs = [
    ("F13P03E1", "F13X03E1"),
    ("F13P03E1", "F13X04E1"),
    ("F13P05E1", "F13X03E1"),
    ("F13P05E1", "F13X04E1")
]
for e_before, e_after in precedence_pairs:
    model5.addConstr(
        gp.quicksum(t * x[e_before, t] for t in timeslots)
        <= gp.quicksum(t * x[e_after, t] for t in timeslots) - 1,
        name=f"precedence_{e_before}_before_{e_after}"
    )

# 23. Misc: H3BFM2E1 must be immediately followed by H3BFM2E2
for t in timeslots[:-1]:
    model5.addConstr(
        x["H3BFM2E1", t] <= x["H3BFM2E2", t+1],
        name=f"consecutive_H3BFM2E1_H3BFM2E2_{t}"
    )
model5.addConstr(
        x["H3BFM2E1", 32] == 0,
        name=f"consecutive_H3BFM2E1_H3BFM2E2_preserve"
    )

# 24. H8B040E1 and H8C001E1 must be at different timeslots
for t in timeslots:
    model5.addConstr(
        x["H8B040E1", t] + x["H8C001E1", t] <= 1,
        name=f"different_time_H8B040E1_H8C001E1_{t}"
    )

# 25. Q33211E1, Q33307E1, and Q33308E1 must be at different timeslots
for t in timeslots:
    model5.addConstr(
            gp.quicksum(x[e, t] for e in ["Q33211E1", "Q33307E1", "Q33308E1"]) <= 1,
            name=f"different_time_Q33211E1_Q33307E1_Q33308E1_{t}"
        )

# 26. B12301E1, B12302E1, B12303E1, and B12320E1 must be at different timeslots
for t in timeslots:
    model5.addConstr(
            gp.quicksum(x[e, t] for e in ["B12301E1", "B12302E1", "B12303E1", "B12320E1"]) <= 1,
            name=f"different_time_B12301E1_B12302E1_B12303E1_B12320E1_{t}"
        )

# 27. B13103E1 and {C13571E1 & C13572E1} must be at different timeslots
for t in timeslots:
    model5.addConstr(
            x["B13103E1", t] + x["C13571E1", t] <= 1,
            name=f"different_time_B13103E1_C13571E1_{t}"
        )

# Objective: Feasible schedule
model5.setObjective(0, GRB.MINIMIZE)

# Solve model
model5.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6127727 rows, 845888 columns and 13047003 nonzeros
Model fingerprint: 0x823126f1
Variable types: 0 continuous, 845888 integer (436288 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 4614338 rows and 34673 columns (presolve time = 5s)...
Presolve removed 5205893 rows and 54887 columns
Presolve time: 5.67s
Presolved: 921834 rows, 791001 columns, 2886261 nonzeros
Variable types: 0 continuous, 791001 integer (414696 binary)
Performing another presolve...
Presolve removed 35291 rows and 13252 columns (presolve time = 5s)...
Presolve removed 62708 rows and 39026 columns
Presolve time: 7.73s
Deterministic concurrent LP optimizer: primal simplex, dual simplex

In [ ]:
schedule_df = get_schedule_model2(x, z)
source5 = plot_source(schedule_df)
make_plot(source5, "Exam Timetable(Model 5)")

rooms_usage_df = get_room_usage_df(x,z)

In [ ]:
# MODEL 5
model5 = gp.Model("Model with miscellaneous")
# model5.setParam("OutputFlag", 0)

# Binary vars x[e,t]
x = model5.addVars(exam_list, timeslots, vtype=GRB.BINARY, name="x")

# Binary vars c[g,t]
c = model5.addVars(groups.keys(), timeslots, vtype=GRB.BINARY, name="c")

# Binary vars  z[e,t,r]
z = model5.addVars(exam_list, timeslots, room_list, vtype=GRB.BINARY, name="z")

# Integer vars p[e,t,r]
p = model5.addVars(exam_list, timeslots, room_list, vtype=GRB.INTEGER, lb=0, name="p")

# 1. Each exam must be scheduled exactly once
for e in exam_list:
    model5.addConstr(
        gp.quicksum(x[e, t] for t in timeslots) == 1, name=f"exam_once_{e}"
    )

# 2. No student may have two exams in the same timeslot
for s, exams_s in students_exams.items():
    for i in range(len(exams_s)):
        for j in range(i + 1, len(exams_s)):
            e1, e2 = exams_s[i], exams_s[j]
            if (e1, e2) in allowed_pairs:
                continue
            for t in timeslots:
                model5.addConstr(
                    x[e1, t] + x[e2, t] <= 1, name=f"no_overlap_{s}_{e1}_{e2}_{t}"
                )

# 3. Exam duration <= timeslot duration
for e in exam_list:
    for t in timeslots:
        if duration[e] > slot_length[t]:
            model5.addConstr(x[e, t] == 0, name=f"forbidden_duration_{e}_{t}")

# 4. Link exams to coincidence group
for g, exams_g in groups.items():
    for e in exams_g:
        for t in timeslots:
            model5.addConstr(x[e, t] <= c[g, t], name=f"link_exam_to_group_{g}_{e}_{t}")

# 5. Coincidence group scheduled exactly once
for g in groups:
    model5.addConstr(
        gp.quicksum(c[g, t] for t in timeslots) == 1, name=f"group_once_{g}"
    )

# 6. Max total capacity per timeslot
for t in timeslots:
    model5.addConstr(
        gp.quicksum(size[e] * x[e, t] for e in exam_list) <= max_capacity,
        name=f"timeslot_capacity_{t}",
    )

# 7. Rooms can be used only if exam is scheduled
for e in exam_list:
    for t in timeslots:
        for r in room_list:
            model5.addConstr(z[e, t, r] <= x[e, t], name=f"room_usage_link_{e}_{t}_{r}")

# 8. Students assigned to a room <= room capacity
for e in exam_list:
    for t in timeslots:
        for r in room_list:
            model5.addConstr(
                p[e, t, r] <= cap[r] * z[e, t, r],
                name=f"room_capacity_link_{e}_{t}_{r}",
            )

# 9. Total exam size distributed across rooms
for e in exam_list:
    for t in timeslots:
        model5.addConstr(
            gp.quicksum(p[e, t, r] for r in room_list) == size[e] * x[e, t],
            name=f"exam_size_{e}_{t}",
        )

# 10. Split exams into no more than 3 rooms
K = 3  # Maximum number of rooms
for e in exam_list:
    for t in timeslots:
        model5.addConstr(
            gp.quicksum(z[e, t, r] for r in room_list) <= K, name=f"max_rooms_{e}_{t}"
        )

# 11. Two exams on same day must have at least 1 gap
for s, exams_s in students_exams.items():
    for d, t_list in day_to_timeslots.items():
        if len(t_list) == 3:
            first_slot = t_list[0]
            middle_slot = t_list[1]
            last_slot = t_list[2]

            # For each pair of non-coincidence exams
            for i in range(len(exams_s)):
                for j in range(i + 1, len(exams_s)):
                    e1, e2 = exams_s[i], exams_s[j]
                    if (e1, e2) in allowed_pairs:
                        continue
                    model5.addConstr(
                        x[e1, middle_slot] + x[e2, first_slot] <= 1,
                        name=f"gap_{s}_{e1}_{e2}_{d}_mid_first",
                    )
                    model5.addConstr(
                        x[e1, middle_slot] + x[e2, middle_slot] <= 1,
                        name=f"gap_{s}_{e1}_{e2}_{d}_mid_mid",
                    )
                    model5.addConstr(
                        x[e1, middle_slot] + x[e2, last_slot] <= 1,
                        name=f"gap_{s}_{e1}_{e2}_{d}_mid_last",
                    )
                    model5.addConstr(
                        x[e1, first_slot] + x[e2, middle_slot] <= 1,
                        name=f"gap_{s}_{e1}_{e2}_{d}_first_mid",
                    )
                    model5.addConstr(
                        x[e1, last_slot] + x[e2, middle_slot] <= 1,
                        name=f"gap_{s}_{e1}_{e2}_{d}_last_mid",
                    )

# 12. Misc: F321Q6E1 must be scheduled on 27th Jan (Fri_1)
model5.addConstr(
    gp.quicksum(x["F321Q6E1", t] for t in day_to_timeslots["Fri_1"]) == 1,
    name="fixed_F321Q6E1_Fri_1"
)

# 13. Misc: F321T6E1 must be scheduled on 30th Jan (Mon_2)
model5.addConstr(
    gp.quicksum(x["F321T6E1", t] for t in day_to_timeslots["Mon_2"]) == 1,
    name="fixed_F321T6E1_Mon_2"
)

# 14. Misc: H21M01E1 must be scheduled on 23rd-24th Jan (Mon_1, Tue_1)
model5.addConstr(
    gp.quicksum(x["H21M01E1", t] for t in day_to_timeslots["Mon_1"] + day_to_timeslots["Tue_1"]) == 1,
    name="fixed_H21M01E1_Mon_1_Tue_1"
)

# 15. Misc: H22M02E1 must be scheduled on 23rd-24th Jan (Mon_1, Tue_1)
model5.addConstr(
    gp.quicksum(x["H22M02E1", t] for t in day_to_timeslots["Mon_1"] + day_to_timeslots["Tue_1"]) == 1,
    name="fixed_H22M02E1_Mon_1_Tue_1"
)

# 16. Misc: H2CM04E1 must be scheduled on 23rd-24th Jan (Mon_1, Tue_1)
model5.addConstr(
    gp.quicksum(x["H2CM04E1", t] for t in day_to_timeslots["Mon_1"] + day_to_timeslots["Tue_1"]) == 1,
    name="fixed_H2CM04E1_Mon_1_Tue_1"
)

# 17. Misc: G13RE2E1 must be scheduled before 30th Jan => timeslots 1-16
model5.addConstr(
    gp.quicksum(x["G13RE2E1", t] for t in timeslots[:16]) == 1,
    name="fixed_G13RE2E1_week1"
)

# 18. Misc: K1AHWAE2  must be scheduled in AM slots only
model5.addConstr(
    gp.quicksum(x["K1AHWAE2", t] for t in am_slots) == 1,
    name="fixed_K1AHWAE2_am"
)

# 19. Misc: H63122E1  must be scheduled in AM slots only
model5.addConstr(
    gp.quicksum(x["H63122E1", t] for t in am_slots) == 1,
    name="fixed_H63122E1_am"
)

# 20. Misc: V13101E1 must be scheduled in any Thursday PM slot (slots 11, 12, 27, 28)
thursday_pm_slots = [11, 12, 27, 28]
model5.addConstr(
    gp.quicksum(x["V13101E1", t] for t in thursday_pm_slots) == 1,
    name="fixed_V13101E1_thu_pm"
)

# 21. Misc: TRENT-B46 unavailable morning of Friday 3rd Feb (slot 29)
for e in exam_list:
    model5.addConstr(
        z[e, 29, "TRENT-B46"] == 0,
        name=f"room_unavail_{e}_29_TRENT-B46"
    )

# 22. Misc: F13P03E1 and F13P05E1 are before F13X03E1 and F13X04E1
precedence_pairs = [
    ("F13P03E1", "F13X03E1"),
    ("F13P03E1", "F13X04E1"),
    ("F13P05E1", "F13X03E1"),
    ("F13P05E1", "F13X04E1")
]
for e_before, e_after in precedence_pairs:
    model5.addConstr(
        gp.quicksum(t * x[e_before, t] for t in timeslots)
        <= gp.quicksum(t * x[e_after, t] for t in timeslots) - 1,
        name=f"precedence_{e_before}_before_{e_after}"
    )

# 23. Misc: H3BFM2E1 must be immediately followed by H3BFM2E2
for t in timeslots[:-1]:
    model5.addConstr(
        x["H3BFM2E1", t] <= x["H3BFM2E2", t+1],
        name=f"consecutive_H3BFM2E1_H3BFM2E2_{t}"
    )
model5.addConstr(
        x["H3BFM2E1", 32] == 0,
        name=f"consecutive_H3BFM2E1_H3BFM2E2_preserve"
    )

# 24. H8B040E1 and H8C001E1 must be at different timeslots
for t in timeslots:
    model5.addConstr(
        x["H8B040E1", t] + x["H8C001E1", t] <= 1,
        name=f"different_time_H8B040E1_H8C001E1_{t}"
    )

# 25. Q33211E1, Q33307E1, and Q33308E1 must be at different timeslots
for t in timeslots:
    model5.addConstr(
            gp.quicksum(x[e, t] for e in ["Q33211E1", "Q33307E1", "Q33308E1"]) <= 1,
            name=f"different_time_Q33211E1_Q33307E1_Q33308E1_{t}"
        )

# 26. B12301E1, B12302E1, B12303E1, and B12320E1 must be at different timeslots
for t in timeslots:
    model5.addConstr(
            gp.quicksum(x[e, t] for e in ["B12301E1", "B12302E1", "B12303E1", "B12320E1"]) <= 1,
            name=f"different_time_B12301E1_B12302E1_B12303E1_B12320E1_{t}"
        )

# 27. B13103E1 and {C13571E1 & C13572E1} must be at different timeslots
for t in timeslots:
    model5.addConstr(
            x["B13103E1", t] + x["C13571E1", t] <= 1,
            name=f"different_time_B13103E1_C13571E1_{t}"
        )
    
# OBJECTIVE FUNCTION

# 1. Room splitting cost
split_cost = gp.quicksum(z[e, t, r] for e in exam_list for t in timeslots for r in room_list)

# # 2. Consecutive conflict cost
# y_conflict = {}
# for s, exams_s in students_exams.items():
#     for i in range(len(exams_s)):
#         for j in range(i + 1, len(exams_s)):
#             e1, e2 = exams_s[i], exams_s[j]
#             if (e1, e2) in allowed_pairs:
#                 continue
#             for t in timeslots[:-1]:  # t and t+1 are consecutive
#                 y_conflict[s, e1, e2, t] = model5.addVar(vtype=GRB.BINARY)

#                 # linearization: y >= x[e1,t] + x[e2,t+1] - 1
#                 model5.addConstr(
#                     y_conflict[s, e1, e2, t] >= x[e1, t] + x[e2, t+1] - 1,
#                     name=f"consecutive_conflict_{s}_{e1}_{e2}_{t}"
#                 )

# conflict_cost = gp.quicksum(y_conflict[s, e1, e2, t]
#                             for (s, e1, e2, t) in y_conflict)

# 3. Earliness penalty
priority_cost = gp.quicksum(
    earliness_priority[e] * t * x[e, t]
    for e in exam_list
    for t in timeslots
)

# Weighted objective 
alpha = 1       # weight for room splitting
beta = 1000     # weight for consecutive exam conflict
gamma = 1       # weight for earliness

model5.setObjective(
    #   alpha * split_cost
    # + beta  * conflict_cost
    gamma * priority_cost
    , GRB.MINIMIZE)

# Solve model
model5.optimize()

Set parameter Username
Set parameter LicenseID to value 2711064
Academic license - for non-commercial use only - expires 2026-09-20
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6127727 rows, 845888 columns and 13047003 nonzeros
Model fingerprint: 0xfb8fad0c
Variable types: 0 continuous, 845888 integer (436288 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [5e+01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 4627524 rows and 54885 columns (presolve time = 5s)...
Presolve removed 5205893 rows and 54887 columns
Presolve time: 5.36s
Presolved: 921834 rows, 791001 columns, 2886261 nonzeros
Variable types: 0 continuous, 791001 integer (414696 binary)
Performing another presolve...
Presolve removed 35221 rows and 13252 columns (presolve time = 5s)...
Pr

In [11]:
schedule_df = get_schedule_model2(x, z)
source5 = plot_source(schedule_df)
make_plot(source5, "Exam Timetable(Model 5)")

rooms_usage_df = get_room_usage_df(x,z)

In [ ]:
# MODEL 5
model5 = gp.Model("Model with miscellaneous")
# model5.setParam("OutputFlag", 0)

# Binary vars x[e,t]
x = model5.addVars(exam_list, timeslots, vtype=GRB.BINARY, name="x")

# Binary vars c[g,t]
c = model5.addVars(groups.keys(), timeslots, vtype=GRB.BINARY, name="c")

# Binary vars  z[e,t,r]
z = model5.addVars(exam_list, timeslots, room_list, vtype=GRB.BINARY, name="z")

# Integer vars p[e,t,r]
p = model5.addVars(exam_list, timeslots, room_list, vtype=GRB.INTEGER, lb=0, name="p")

# 1. Each exam must be scheduled exactly once
for e in exam_list:
    model5.addConstr(
        gp.quicksum(x[e, t] for t in timeslots) == 1, name=f"exam_once_{e}"
    )

# 2. No student may have two exams in the same timeslot
for s, exams_s in students_exams.items():
    for i in range(len(exams_s)):
        for j in range(i + 1, len(exams_s)):
            e1, e2 = exams_s[i], exams_s[j]
            if (e1, e2) in allowed_pairs:
                continue
            for t in timeslots:
                model5.addConstr(
                    x[e1, t] + x[e2, t] <= 1, name=f"no_overlap_{s}_{e1}_{e2}_{t}"
                )

# 3. Exam duration <= timeslot duration
for e in exam_list:
    for t in timeslots:
        if duration[e] > slot_length[t]:
            model5.addConstr(x[e, t] == 0, name=f"forbidden_duration_{e}_{t}")

# 4. Link exams to coincidence group
for g, exams_g in groups.items():
    for e in exams_g:
        for t in timeslots:
            model5.addConstr(x[e, t] <= c[g, t], name=f"link_exam_to_group_{g}_{e}_{t}")

# 5. Coincidence group scheduled exactly once
for g in groups:
    model5.addConstr(
        gp.quicksum(c[g, t] for t in timeslots) == 1, name=f"group_once_{g}"
    )

# 6. Max total capacity per timeslot
for t in timeslots:
    model5.addConstr(
        gp.quicksum(size[e] * x[e, t] for e in exam_list) <= max_capacity,
        name=f"timeslot_capacity_{t}",
    )

# 7. Rooms can be used only if exam is scheduled
for e in exam_list:
    for t in timeslots:
        for r in room_list:
            model5.addConstr(z[e, t, r] <= x[e, t], name=f"room_usage_link_{e}_{t}_{r}")

# 8. Students assigned to a room <= room capacity
for e in exam_list:
    for t in timeslots:
        for r in room_list:
            model5.addConstr(
                p[e, t, r] <= cap[r] * z[e, t, r],
                name=f"room_capacity_link_{e}_{t}_{r}",
            )

# 9. Total exam size distributed across rooms
for e in exam_list:
    for t in timeslots:
        model5.addConstr(
            gp.quicksum(p[e, t, r] for r in room_list) == size[e] * x[e, t],
            name=f"exam_size_{e}_{t}",
        )

# 10. Split exams into no more than 3 rooms
K = 3  # Maximum number of rooms
for e in exam_list:
    for t in timeslots:
        model5.addConstr(
            gp.quicksum(z[e, t, r] for r in room_list) <= K, name=f"max_rooms_{e}_{t}"
        )

# 11. Two exams on same day must have at least 1 gap
for s, exams_s in students_exams.items():
    for d, t_list in day_to_timeslots.items():
        if len(t_list) == 3:
            first_slot = t_list[0]
            middle_slot = t_list[1]
            last_slot = t_list[2]

            # For each pair of non-coincidence exams
            for i in range(len(exams_s)):
                for j in range(i + 1, len(exams_s)):
                    e1, e2 = exams_s[i], exams_s[j]
                    if (e1, e2) in allowed_pairs:
                        continue
                    model5.addConstr(
                        x[e1, middle_slot] + x[e2, first_slot] <= 1,
                        name=f"gap_{s}_{e1}_{e2}_{d}_mid_first",
                    )
                    model5.addConstr(
                        x[e1, middle_slot] + x[e2, middle_slot] <= 1,
                        name=f"gap_{s}_{e1}_{e2}_{d}_mid_mid",
                    )
                    model5.addConstr(
                        x[e1, middle_slot] + x[e2, last_slot] <= 1,
                        name=f"gap_{s}_{e1}_{e2}_{d}_mid_last",
                    )
                    model5.addConstr(
                        x[e1, first_slot] + x[e2, middle_slot] <= 1,
                        name=f"gap_{s}_{e1}_{e2}_{d}_first_mid",
                    )
                    model5.addConstr(
                        x[e1, last_slot] + x[e2, middle_slot] <= 1,
                        name=f"gap_{s}_{e1}_{e2}_{d}_last_mid",
                    )

# 12. Misc: F321Q6E1 must be scheduled on 27th Jan (Fri_1)
model5.addConstr(
    gp.quicksum(x["F321Q6E1", t] for t in day_to_timeslots["Fri_1"]) == 1,
    name="fixed_F321Q6E1_Fri_1"
)

# 13. Misc: F321T6E1 must be scheduled on 30th Jan (Mon_2)
model5.addConstr(
    gp.quicksum(x["F321T6E1", t] for t in day_to_timeslots["Mon_2"]) == 1,
    name="fixed_F321T6E1_Mon_2"
)

# 14. Misc: H21M01E1 must be scheduled on 23rd-24th Jan (Mon_1, Tue_1)
model5.addConstr(
    gp.quicksum(x["H21M01E1", t] for t in day_to_timeslots["Mon_1"] + day_to_timeslots["Tue_1"]) == 1,
    name="fixed_H21M01E1_Mon_1_Tue_1"
)

# 15. Misc: H22M02E1 must be scheduled on 23rd-24th Jan (Mon_1, Tue_1)
model5.addConstr(
    gp.quicksum(x["H22M02E1", t] for t in day_to_timeslots["Mon_1"] + day_to_timeslots["Tue_1"]) == 1,
    name="fixed_H22M02E1_Mon_1_Tue_1"
)

# 16. Misc: H2CM04E1 must be scheduled on 23rd-24th Jan (Mon_1, Tue_1)
model5.addConstr(
    gp.quicksum(x["H2CM04E1", t] for t in day_to_timeslots["Mon_1"] + day_to_timeslots["Tue_1"]) == 1,
    name="fixed_H2CM04E1_Mon_1_Tue_1"
)

# 17. Misc: G13RE2E1 must be scheduled before 30th Jan => timeslots 1-16
model5.addConstr(
    gp.quicksum(x["G13RE2E1", t] for t in timeslots[:16]) == 1,
    name="fixed_G13RE2E1_week1"
)

# 18. Misc: K1AHWAE2  must be scheduled in AM slots only
model5.addConstr(
    gp.quicksum(x["K1AHWAE2", t] for t in am_slots) == 1,
    name="fixed_K1AHWAE2_am"
)

# 19. Misc: H63122E1  must be scheduled in AM slots only
model5.addConstr(
    gp.quicksum(x["H63122E1", t] for t in am_slots) == 1,
    name="fixed_H63122E1_am"
)

# 20. Misc: V13101E1 must be scheduled in any Thursday PM slot (slots 11, 12, 27, 28)
thursday_pm_slots = [11, 12, 27, 28]
model5.addConstr(
    gp.quicksum(x["V13101E1", t] for t in thursday_pm_slots) == 1,
    name="fixed_V13101E1_thu_pm"
)

# 21. Misc: TRENT-B46 unavailable morning of Friday 3rd Feb (slot 29)
for e in exam_list:
    model5.addConstr(
        z[e, 29, "TRENT-B46"] == 0,
        name=f"room_unavail_{e}_29_TRENT-B46"
    )

# 22. Misc: F13P03E1 and F13P05E1 are before F13X03E1 and F13X04E1
precedence_pairs = [
    ("F13P03E1", "F13X03E1"),
    ("F13P03E1", "F13X04E1"),
    ("F13P05E1", "F13X03E1"),
    ("F13P05E1", "F13X04E1")
]
for e_before, e_after in precedence_pairs:
    model5.addConstr(
        gp.quicksum(t * x[e_before, t] for t in timeslots)
        <= gp.quicksum(t * x[e_after, t] for t in timeslots) - 1,
        name=f"precedence_{e_before}_before_{e_after}"
    )

# 23. Misc: H3BFM2E1 must be immediately followed by H3BFM2E2
for t in timeslots[:-1]:
    model5.addConstr(
        x["H3BFM2E1", t] <= x["H3BFM2E2", t+1],
        name=f"consecutive_H3BFM2E1_H3BFM2E2_{t}"
    )
model5.addConstr(
        x["H3BFM2E1", 32] == 0,
        name=f"consecutive_H3BFM2E1_H3BFM2E2_preserve"
    )

# 24. H8B040E1 and H8C001E1 must be at different timeslots
for t in timeslots:
    model5.addConstr(
        x["H8B040E1", t] + x["H8C001E1", t] <= 1,
        name=f"different_time_H8B040E1_H8C001E1_{t}"
    )

# 25. Q33211E1, Q33307E1, and Q33308E1 must be at different timeslots
for t in timeslots:
    model5.addConstr(
            gp.quicksum(x[e, t] for e in ["Q33211E1", "Q33307E1", "Q33308E1"]) <= 1,
            name=f"different_time_Q33211E1_Q33307E1_Q33308E1_{t}"
        )

# 26. B12301E1, B12302E1, B12303E1, and B12320E1 must be at different timeslots
for t in timeslots:
    model5.addConstr(
            gp.quicksum(x[e, t] for e in ["B12301E1", "B12302E1", "B12303E1", "B12320E1"]) <= 1,
            name=f"different_time_B12301E1_B12302E1_B12303E1_B12320E1_{t}"
        )

# 27. B13103E1 and {C13571E1 & C13572E1} must be at different timeslots
for t in timeslots:
    model5.addConstr(
            x["B13103E1", t] + x["C13571E1", t] <= 1,
            name=f"different_time_B13103E1_C13571E1_{t}"
        )
    
# OBJECTIVE FUNCTION

# 1. Room splitting cost
split_cost = gp.quicksum(z[e, t, r] for e in exam_list for t in timeslots for r in room_list)

# 2. Earliness penalty
priority_cost = gp.quicksum(
    earliness_priority[e] * t * x[e, t]
    for e in exam_list
    for t in timeslots
)

# Weighted objective 
alpha = 1       # weight for room splitting
gamma = 0.3       # weight for earliness

model5.setObjective(
    gamma * priority_cost
    + alpha * split_cost
    , GRB.MINIMIZE)

# Solve model
model5.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6127727 rows, 845888 columns and 13047003 nonzeros
Model fingerprint: 0xebc7e7a2
Variable types: 0 continuous, 845888 integer (436288 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e+00, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 4627524 rows and 54885 columns (presolve time = 5s)...
Presolve removed 5205893 rows and 54887 columns
Presolve time: 5.45s
Presolved: 921834 rows, 791001 columns, 2886261 nonzeros
Variable types: 0 continuous, 791001 integer (414696 binary)
Performing another presolve...
Presolve removed 35221 rows and 13252 columns (presolve time = 5s)...
Presolve removed 62478 rows and 38876 columns
Presolve time: 7.54s
Deterministic concurrent LP optimizer: primal simplex, dual simplex

In [13]:
schedule_df = get_schedule_model2(x, z)
source5 = plot_source(schedule_df)
make_plot(source5, "Exam Timetable(Model 5)")

rooms_usage_df = get_room_usage_df(x,z)